In [1]:
#python_bitbankccのパッケージをインポート
#cloud9で起動するときのコマンド
#jupyter notebook --ip $IP --port $PORT --no-browser
import python_bitbankcc 
import datetime
import os 
import time
import numpy as np
import pandas as pd
import sys
from dateutil.relativedelta import relativedelta
#トレンドラインを引くため
from scipy.stats import linregress

In [2]:
#機械学習用のモジュール
import matplotlib as mpl
mpl.use('Agg')
import matplotlib.pyplot as plt
from sklearn import linear_model
% matplotlib inline
from __future__ import print_function
import copy
import matplotlib
matplotlib.style.use('ggplot')
import time
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [3]:
# public API classのオブジェクトを取得
pub = python_bitbankcc.public()

In [4]:
#APIから基本データの取得、dataframeへ挿入
def get_candle(trade_name,span,back_day):
    # ロウソク足データを取得
    value = pub.get_candlestick( trade_name,span, back_day )
    candle = value['candlestick'][0]
    #print(candle['ohlcv'][0])
    dataframe = pd.DataFrame(candle['ohlcv'],columns=["Open", "High","Low","Close","Volume","Timestamp"])
    return dataframe


In [5]:
#指定した日から今日までの基本データをdataframeにして取得
#back_day,todayはdatetime
# span = ['1min', '5min', '15min', '30min', '1hour', '4hour', '8hour', '12hour', '1day', '1week']
#trade_name = ['btc_jpy', 'xrp_jpy', 'ltc_btc', 'eth_btc', 'mona_jpy', 'mona_btc', 'bcc_jpy', 'bcc_btc']

def make_df(trade_name,span,back_day,today):
    i = 1
    if (span =='1min')or(span== '5min')or(span== '15min')or(span== '30min')or(span== '1hour'):
        #基準が９時なので、そこを合わせてあげる
        if 0 <= today.hour < 9:
            k = 1
        elif 9 <= today.hour <= 23:
            k = 0

        while back_day <= today - datetime.timedelta(days=k):
            if i == 1:
                df1 = get_candle(trade_name,span,datetime.datetime.strftime(back_day, '%Y%m%d'))
                back_day = back_day + datetime.timedelta(days=1)
                #print(back_day,len(df1))
                i += 1
            else:
                df2 = get_candle(trade_name,span,datetime.datetime.strftime(back_day, '%Y%m%d'))
                df1 = pd.concat([df1, df2])
                back_day = back_day + datetime.timedelta(days=1)
                #print(back_day,len(df1))
                i += 1
    else:
        
        today = datetime.date(today.year, today.month, today.day)
        back_day = datetime.date(back_day.year, back_day.month, back_day.day)


        while back_day <= today:
            if i == 1:
                df1 = get_candle(trade_name,span,datetime.datetime.strftime(back_day, '%Y'))
                back_day = back_day + relativedelta(years=1)
                i += 1
            else:
                df2 = get_candle(trade_name,span,datetime.datetime.strftime(back_day, '%Y'))
                df1 = pd.concat([df1, df2])
                back_day = back_day +  relativedelta(years=1)
                i += 1
            

    return df1



In [6]:
def read_date(x):
    return datetime.datetime.fromtimestamp(x/1000)


In [7]:

# 単純移動平均（SMA）を取得する関数
#上記のdfデータフレーム
#numいくつの平均を求めるか
def getMA( df,num ):
 
    tmp = []
    avg = np.array([])
    for i in range(len(df) - num + 1):
        for j in range(num):
            #print(df['Close'][i+j])
            tmp.append( df['Close'][i+j])
            
         # 平均値計算
        value = np.average(tmp)
        avg = np.append(avg,value)
        tmp = []
    
    return avg

In [8]:
#ゴールデン・デッドクロス判定
def golden_dead(data_df1):
    golden=np.zeros(len(data_df1['Cross']),dtype=int)
    dead=np.zeros(len(data_df1['Cross']),dtype=int)
    for i in range(len(data_df1['Cross'])-1):
        x = data_df1['Cross'][i]
        y = data_df1['Cross'][i+1]
        if ((x <= 0) & (y<=0)) |((x >= 0) & (y>=0)):
            pass
        elif ((x <= 0) & (y>=0)) :
            golden=np.insert(golden,i+1,1)
            golden=np.delete(golden,i+2)
        elif ((x >= 0) & (y<=0)) :
            dead=np.insert(dead,i+1,1)
            dead=np.delete(dead,i+2)

    return golden,dead
    

In [9]:
#上下判定
def up_down(data_df1):
    up=np.zeros(len(data_df1['Close']),dtype=int)
    mid_up=np.zeros(len(data_df1['Close']),dtype=int)
    down=np.zeros(len(data_df1['Close']),dtype=int)
    mid_down=np.zeros(len(data_df1['Close']),dtype=int)
    for i in range(len(data_df1['Close'])-1):
        x = data_df1['Close'][i]
        y = data_df1['Close'][i+1]
#         if (0<=(y-x)<=0.5):
#             mid_up=np.insert(mid_up,i+1,1)
#             min_up=np.delete(mid_up,i+2)
#         elif (-0.5<=(y-x)<0):
#             mid_down=np.insert(mid_down,i+1,1)
#             min_down=np.delete(mid_down,i+2)
        if ((y-x)>0):
            up=np.insert(up,i+1,1)
            up=np.delete(up,i+2)
        elif ((y-x)<=0) :
            down=np.insert(down,i+1,1)
            down=np.delete(down,i+2)

    return up,down

In [10]:
#関数の使い方
today = datetime.datetime.today() 
back_day = today - datetime.timedelta(days=400)
trade_name = 'xrp_jpy'
span = '12hour'
num = 5

In [11]:
#基本データの取得
old = time.time()
data_df1 =  make_df(trade_name,span,back_day,today)
data_df1['Timestamp'] = data_df1['Timestamp'].apply(read_date)
#indexの割り振り
data_df1 = data_df1.reset_index()
#df1の値は全てstrなのでTimestamp以外floatにキャストする
#Timestampは文字列のままでいいので削除することを指定
execlude = ['Timestamp']
cast = [col for col in data_df1.columns if col not in execlude]
for item in cast:
    data_df1 = data_df1.astype({item: float})
print(data_df1)

     index    Open    High     Low   Close        Volume           Timestamp
0      0.0  34.375  34.375  34.375  34.375  0.000000e+00 2017-05-24 21:00:00
1      1.0  34.375  35.500  34.000  34.000  3.072000e+03 2017-05-25 09:00:00
2      2.0  34.000  36.000  31.000  35.000  6.133000e+03 2017-05-25 21:00:00
3      3.0  35.000  40.000  30.200  40.000  9.901000e+03 2017-05-26 09:00:00
4      4.0  40.000  40.000  30.500  36.000  1.254100e+04 2017-05-26 21:00:00
5      5.0  36.000  36.000  20.000  20.000  9.026198e+04 2017-05-27 09:00:00
6      6.0  20.000  36.000  20.000  27.000  4.969786e+03 2017-05-27 21:00:00
7      7.0  27.000  29.000  27.000  29.000  1.358398e+04 2017-05-28 09:00:00
8      8.0  29.000  29.000  20.500  29.000  7.202677e+03 2017-05-28 21:00:00
9      9.0  29.000  29.550  25.500  25.500  1.124225e+04 2017-05-29 09:00:00
10    10.0  25.500  27.900  25.500  25.501  1.260000e+03 2017-05-29 21:00:00
11    11.0  25.501  27.500  25.501  27.000  4.440000e+02 2017-05-30 09:00:00

In [12]:
# #移動平均線np
# MA5 = getMA( data_df1,num)
# MA25 = getMA( data_df1,num+20)
# zero5 = np.zeros(num-1,dtype = float )
# zero25 = np.zeros(num+20-1,dtype = float )
# MA5 = np.insert(MA5, 0, zero5)
# MA25 = np.insert(MA25, 0, zero25)
# print(MA25,MA5)
# print(MA25.shape,type(MA25),MA5.shape,type(MA5))

In [13]:
# #自作変数を作成する
# data_df1['High-Low']=data_df1['High']-data_df1['Low']
# data_df1['Close-Open']=data_df1['Close']-data_df1['Open']
# data_df1['Similarity'] = data_df1['Close-Open'] / (data_df1['High-Low'] + 0.000001)
# data_df1['(High-Low)*Volume '] = data_df1['High-Low']*data_df1['Volume']
# data_df1['(Close-Open)*Volume '] = data_df1['Close-Open']*data_df1['Volume']
# data_df1['High-Low']
# data_df1['MA5'] = MA5
# data_df1['MA25'] = MA25
# data_df1['Cross'] = MA5 - MA25
# data_df1['index'] = data_df1.index + 1

In [14]:
# data_df1['golden'],data_df1['dead']  = golden_dead(data_df1)

In [15]:
# #data_df1['up'],data_df1['mid_up'],data_df1['min_down'],data_df1['down'] = up_down(data_df1)
# a,b = up_down(data_df1)
# # print(len(a),len(b))
# data_df1['up'],data_df1['down'] = up_down(data_df1)


In [16]:
# #data作成までの時間
# print('かかった時間：{0}'.format(time.time()-old))
# #作成変数の表示
# data_df1.loc[len(data_df1)-9:]

In [17]:
#次のCloseを予測する機械学習
#使わない変数を除外
exe_cols = ['index','Timestamp']
feature_cols = [col for col in data_df1.columns if col not in exe_cols]
print(feature_cols)
#機械学習用にnp配列に変換
data_np= np.array(data_df1[feature_cols])
print(data_np.shape)

['Open', 'High', 'Low', 'Close', 'Volume']
(1000, 5)


In [18]:
Y = np.array(data_df1['Close'])#1次元
Y = np.delete(Y,0)
#print(Y,Y.shape)
X = data_np[:len(Y),0:]
#print(X[:],X.shape)

[ 34.     35.     40.     36.     20.     27.     29.     29.     25.5
  25.501  27.     27.9    26.     29.     35.     37.8    36.483  36.4
  33.     31.     32.999  35.8    32.22   32.218  31.69   31.5    31.2
  31.     32.1    31.799  32.03   31.8    29.801  29.5    28.901  29.999
  28.1    31.498  29.773  30.     30.702  30.957  28.66   27.579  28.1
  28.351  29.12   29.645  28.55   29.51   30.1    29.138  35.152  36.
  32.9    30.5    30.61   31.289  31.3    32.6    33.01   31.78   31.19
  30.7    29.9    29.079  28.95   29.708  29.34   30.35   28.469  28.62
  29.486  28.5    28.565  27.8    27.694  28.4    28.125  28.49   28.5
  28.485  28.3    28.12   28.238  28.319  27.21   26.569  25.72   25.869
  26.596  26.14   25.615  21.631  21.324  20.     21.554  21.933  23.
  22.3    21.841  20.514  19.66   19.438  15.516  16.819  16.796  19.3
  19.79   20.52   20.5    19.11   19.579  21.     21.279  19.951  20.41
  21.64   21.654  21.85   21.725  21.532  19.896  19.819  19.777  19.329

In [19]:
# 学習では、29/30を使うものとします。これは情報の偏りを防ぐためのものであり、全体でも構いません
L = int(len(X)//(200/199))
print(L)
train_x = X[10:L,:]
train_y = Y[10:L]

994


In [20]:
# 残りの全てをテストデータとします
test_x = X[L:len(X),:]
test_y = Y[L:len(Y)]

In [21]:
#ニューラルネットワークを組む時のライブラリ
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error
import tensorflow as tf

C:\Users\ibuki\Anaconda3\envs\py36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [22]:
df_2 = data_df1[:len(Y)]
del df_2['index']
del df_2['Timestamp']
print(len(Y),len(df_2))
df_2['Close_tomorrow'] = Y
print(df_2[len(df_2)-9:])
# データセットのサイズを確認
print(df_2.shape[0])
print(df_2.shape[1])

999 999
       Open    High     Low   Close        Volume  Close_tomorrow
990  65.817  67.160  60.492  65.140  1.890289e+08          63.045
991  65.140  65.500  62.294  63.045  7.882499e+07          57.600
992  63.045  64.648  57.200  57.600  1.317778e+08          60.390
993  57.598  61.800  57.000  60.390  1.303197e+08          60.001
994  60.391  60.888  58.589  60.001  7.354091e+07          60.230
995  60.003  62.701  59.800  60.230  8.873191e+07          60.018
996  60.230  61.181  59.611  60.018  4.175793e+07          58.840
997  60.008  60.512  58.011  58.840  6.402522e+07          58.900
998  58.803  59.997  57.510  58.900  4.667886e+07          58.859
999
6


C:\Users\ibuki\Anaconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [23]:
n = df_2.shape[0]
p = df_2.shape[1]
 
# 訓練データとテストデータへ切り分け
train_start = 30
train_end = int(np.floor(0.95*n))
test_start = train_end + 1
test_end = n
data_train = df_2[train_start: train_end]
data_test = df_2[test_start:test_end]

In [26]:
print(data_train[:5])

print(data_test[len(data_test)-5:])

      Open    High     Low   Close       Volume  Close_tomorrow
30  32.100  32.998  31.000  31.799   74039.2787          32.030
31  31.799  32.998  31.751  32.030  143676.7695          31.800
32  32.030  32.537  31.800  31.800  234524.3268          29.801
33  31.800  32.200  29.500  29.801   38337.6534          29.500
34  29.504  30.652  29.500  29.500  148610.7275          28.901
       Open    High     Low   Close        Volume  Close_tomorrow
994  60.391  60.888  58.589  60.001  7.354091e+07          60.230
995  60.003  62.701  59.800  60.230  8.873191e+07          60.018
996  60.230  61.181  59.611  60.018  4.175793e+07          58.840
997  60.008  60.512  58.011  58.840  6.402522e+07          58.900
998  58.803  59.997  57.510  58.900  4.667886e+07          58.859


In [27]:
# データの正規化
scaler = MinMaxScaler()
print(len(data_train))
scaler.fit(data_train)
data_train_norm = scaler.transform(data_train)
data_test_norm = scaler.transform(data_test)
print(data_train_norm.shape,data_test_norm.shape)

919
(919, 6) (49, 6)


In [28]:
# 特徴量とターゲットへ切り分け
print(type(data_train_norm))
X_train = data_train_norm[:,:data_train_norm.shape[1]-1]
y_train = data_train_norm[:,data_train_norm.shape[1]-1:]
X_test = data_test_norm[:, :data_train_norm.shape[1]-1]
y_test = data_test_norm[:, data_train_norm.shape[1]-1:]
print(X_train.shape,y_train.shape,X_test.shape,y_test.shape)

# y_test=np.insert(y_test,0,0)
# y_test=np.delete(y_test,len(y_test)-1)

# y_test=y_test.reshape(y_test.shape[0],1)
# #print(X_train[:,[3]])

# print(X_test[:,[3]]-y_test)

<class 'numpy.ndarray'>
(919, 5)
(919, 1) (49, 5) (49, 1)


In [56]:
y_test = y_test.reshape(y_test.shape[0], 1)
test_inv = np.concatenate(( X_test,y_test), axis=1)
test_inv = scaler.inverse_transform(test_inv)

In [42]:
print(data_test[:2],len(y_test))
 
# 正規化後のテストデータ
#print(y_test, X_test[0])
 
# 正規化から戻したデータ
#print(test_inv[:2])

      Open    High   Low   Close        Volume  Close_tomorrow
950  29.22  29.759  28.3  29.240  6.368997e+07          28.541
951  29.24  29.484  28.3  28.541  5.413600e+07          30.021 49


In [43]:
n_stocks = X_train.shape[1]
y_stocks = y_train.shape[1]
# ニューロンの数を設定
n_neurons_1 = 256
n_neurons_2 = 128
 
# セッションの開始
net = tf.InteractiveSession()
 
# プレースホルダーの作成
X = tf.placeholder(dtype=tf.float32, shape=[None,n_stocks])
Y = tf.placeholder(dtype=tf.float32, shape=[None,y_stocks])
 
# 初期化
sigma = 1
weight_initializer = tf.variance_scaling_initializer(mode="fan_avg", distribution="uniform", scale=sigma)
bias_initializer = tf.zeros_initializer()
 

C:\Users\ibuki\Anaconda3\envs\py36\lib\site-packages\tensorflow\python\client\session.py:1645: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [44]:
#隠れ層の重み
W_hidden_1 = tf.Variable(weight_initializer([n_stocks, n_neurons_1]))
bias_hidden_1 = tf.Variable(bias_initializer([n_neurons_1]))
W_hidden_2 = tf.Variable(weight_initializer([n_neurons_1, n_neurons_2]))
bias_hidden_2 = tf.Variable(bias_initializer([n_neurons_2]))
 

In [45]:
#出力の重み
W_out = tf.Variable(weight_initializer([n_neurons_2, 1]))
bias_out = tf.Variable(bias_initializer([1]))

In [46]:
# 隠れ層の設定（ReLU＝活性化関数）
hidden_1 = tf.nn.relu(tf.add(tf.matmul(X, W_hidden_1), bias_hidden_1))
hidden_2 = tf.nn.relu(tf.add(tf.matmul(hidden_1, W_hidden_2), bias_hidden_2))




# 出力層の設定
out = tf.transpose(tf.add(tf.matmul(hidden_2, W_out), bias_out))

In [47]:
# コスト関数
mse = tf.reduce_mean(tf.squared_difference(out, Y))
 
# 最適化関数
opt = tf.train.AdamOptimizer().minimize(mse)
 
# 初期化
net.run(tf.global_variables_initializer())

In [48]:
# ニューラルネットワークの設定
batch_size = 128
mse_train = []
mse_test = []
 
# 訓練開始！500回の反復処理
epochs = 500
for e in range(epochs):
    net.run(opt, feed_dict={X: X_train, Y: y_train})

In [49]:
# テストデータで予測
pred_test = net.run(out, feed_dict={X: X_test})
 
# 予測データの最初の2つを表示
print(type(pred_test))
print(pred_test.T.shape,X_test.shape)
print(pred_test[0:2])

<class 'numpy.ndarray'>
(49, 1) (49, 5)
[[0.12701592 0.12694934 0.12668982 0.12714954 0.12639007 0.12707894
  0.12693325 0.12621802 0.12627167 0.12601644 0.12662242 0.12644151
  0.12688701 0.12647864 0.12706648 0.12718968 0.12692168 0.12784407
  0.13005325 0.12980911 0.12648953 0.13108234 0.12587336 0.12587428
  0.12688649 0.12460575 0.12610866 0.1284188  0.12585568 0.13016786
  0.12480951 0.1252613  0.12684754 0.12632492 0.12690394 0.12595552
  0.12452137 0.12406452 0.12392084 0.1260415  0.12639719 0.12688792
  0.12484908 0.12473513 0.12675843 0.12657936 0.1273195  0.12673804
  0.12706953]]


In [60]:
# 予測値をテストデータに戻そう（値も正規化からインバース）
print(X_test.shape,pred_test.T.shape,type(pred_test))
pred_tested = np.concatenate(( X_test,pred_test.T), axis=1)
#print(pred_tested)
pred_inv = scaler.inverse_transform(pred_tested)
print(test_inv-pred_inv)

(49, 5) (49, 1) <class 'numpy.ndarray'>
[[  0.           0.           0.           0.           0.
  -33.20676208]
 [  0.           0.           0.           0.           0.
  -31.70252861]
 [  0.           0.           0.           0.           0.
  -31.13206798]
 [  0.           0.           0.           0.           0.
  -30.54039715]
 [  0.           0.           0.           0.           0.
  -30.54196314]
 [  0.           0.           0.           0.           0.
  -30.84069926]
 [  0.           0.           0.           0.           0.
  -30.46767092]
 [  0.           0.           0.           0.           0.
  -30.02834013]
 [  0.           0.           0.           0.           0.
  -30.20686575]
 [  0.           0.           0.           0.           0.
  -29.98396717]
 [  0.           0.           0.           0.           0.
  -30.21353621]
 [  0.           0.           0.           0.           0.
  -31.29868604]
 [  0.           0.           0.           0.           0.
 

In [61]:
# 元データの最後尾
data_test.tail()


,Open,High,Low,Close,Volume,Close_tomorrow
994,60.391,60.888,58.589,60.001,7.354091e+07,60.230
995,60.003,62.701,59.800,60.230,8.873191e+07,60.018
996,60.230,61.181,59.611,60.018,4.175793e+07,58.840
997,60.008,60.512,58.011,58.840,6.402522e+07,58.900
998,58.803,59.997,57.510,58.900,4.667886e+07,58.859


In [62]:
# テストデータの最後のデータ（正規化を戻した後）
print(test_inv[len(data_test)-2:])
 
# モデルが予測したデータ
print(pred_inv[len(data_test)-2:,:])

[[6.00080000e+01 6.05120000e+01 5.80110000e+01 5.88400000e+01
  6.40252201e+07 5.89000000e+01]
 [5.88030000e+01 5.99970000e+01 5.75100000e+01 5.89000000e+01
  4.66788622e+07 5.88590000e+01]]
[[6.00080000e+01 6.05120000e+01 5.80110000e+01 5.88400000e+01
  6.40252201e+07 6.16466193e+01]
 [5.88030000e+01 5.99970000e+01 5.75100000e+01 5.89000000e+01
  4.66788622e+07 6.17672769e+01]]


In [63]:
print(test_inv[len(data_test)-10:,n_stocks:])

[[65.14 ]
 [63.045]
 [57.6  ]
 [60.39 ]
 [60.001]
 [60.23 ]
 [60.018]
 [58.84 ]
 [58.9  ]
 [58.859]]


In [64]:
# Plot拡大
plt.plot(range(0,10), test_inv[len(data_test)-10:,n_stocks:], label='Actual price', color='blue', marker = 'o')
plt.plot(range(0,10), pred_inv[len(data_test)-10:,n_stocks:], label='Predicted price', color='red', marker ='x')
plt.xlabel('1Hours')
plt.ylabel('Price ($)')
plt.title('xrp_jpy')
plt.grid(True)
plt.legend()
plt.savefig('NN_xpr_jpy.png')
plt.close()
plt.show()

In [55]:
print(y_test.shape,pred_inv.shape)

(49, 1) (49, 6)
